In [1]:
# copy dataset
#!wget --no-check-certificate \
#  https://drive.google.com/file/d/1A8g74ohdb_5d2fPjc72yF7GxufE9GRcu/view?usp=sharing \
#  -O /tmp/set_1.pkl

#!wget --no-check-certificate \
#  https://drive.google.com/file/d/1-mPKMpphaKqtT26ZzbR5hCHGedkNyAf1/view?usp=sharing \
#  -O /tmp/Image_captioning_Dataset.zip

#!pip install PyDrive

#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#from google.colab import auth
#from oauth2client.client import GoogleCredentials

#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)

# https://drive.google.com/file/d/1uaD3aDq1Xm7jFII6NYhbegtyREC6NAq-/view?usp=sharing
#downloaded = drive.CreateFile({'id':"1uaD3aDq1Xm7jFII6NYhbegtyREC6NAq-"})   # replace the id with id of file you want to access
#downloaded.GetContentFile('set_1.pkl')        # replace the file name with your file

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [27]:
import pandas as pd
import pickle
import os

FILE_PATH = "/content/gdrive/MyDrive/Datasets"
pickle_name = os.path.join(FILE_PATH, "set_1.pkl")

# Method 1
objects = pd.read_pickle(pickle_name)
df = pd.DataFrame(objects)
print(type(df))
print(df)

# Method 2
pickle_file = open(pickle_name, "rb")
objects = []

while True:
    try:
        objects.append(pickle.load(pickle_file))
    except EOFError:
        break

pickle_file.close()
df = pd.DataFrame(objects)
print(type(df))
print(df)

<class 'pandas.core.frame.DataFrame'>
                                                       0
0      2089122314_40d5739aef.jpg#3\tA fluffy , tri-co...
1      1295671216_cde1b9c9d1.jpg#0\tA crowd of person...
2      3183195185_cd0ff994a1.jpg#4\tThe man and boy s...
3      2949353587_64c54e9589.jpg#1\tA brown dog be pl...
4      361092202_3d70144ebd.jpg#2\tA man hike with a ...
...                                                  ...
24995  3503623999_bbd5dcfb18.jpg#0\tA man in a crowd ...
24996  542405691_0594b1ce72.jpg#0\tA boy kick a ball ...
24997  1244485675_822e6efe60.jpg#1\tA boy look at his...
24998  475778645_65b7343c47.jpg#4\tA small child clim...
24999  381052465_722e00807b.jpg#4\tPeople put out a f...

[25000 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>
                                               0      ...                                              24999
0  2089122314_40d5739aef.jpg#3\tA fluffy , tri-co...  ...  381052465_722e00807b.jpg#4\tPeople put out a f